# Parking in Seattle

Driving in Seattle is quickly becoming very similar to driving in cities like San Francisco, Silicon Valley or Los Angeles: more and more companies choose to settle or open their offices in Seattle so they can tap into the tech community that Seattle has to offer. With that, parking in Seattle is getting harder by day.

Paid Parking Occupancy dataset provided by the City of Seattle Department of Transportation provides a view into around 300 million parking transactions annually from around 12 thousands parking spots on roughly 1,500 block faces. The dataset does not include any transaction for Sundays as there is no paid parking. Most of the parking spots have a 2-hour limit.

## Load the modules

First, we'll load the necessary modules and instantiate the `BlazingContext()`.

In [ ]:
import cupy as cp
import numpy as np
import cudf
import dask_cudf
import cugraph
import cuspatial
import datetime as dttm
from cuml.preprocessing import train_test_split
from dask_cuda import LocalCUDACluster
from dask.distributed import Client
from blazingsql import BlazingContext
from os import listdir

bc = BlazingContext()

_ = bc.s3(
    'bsql'
    , bucket_name = 'bsql'
)

# Create the tables

First, we read the data.

In [ ]:
transactions_partitions_cnt = 40
transactions_path = 's3://bsql/data/seattle_parking/parking_MayJun2019.parquet/partition_idx={partition}/'
transactions_parq = [transactions_path.format(partition=p) for p in range(transactions_partitions_cnt)]

locations_parq = 's3://bsql/data/seattle_parking/parking_locations.parquet/'

In [ ]:
bc.create_table('parking_transactions', transactions_parq)
bc.create_table('parking_locations', locations_parq)

parking_locations = bc.sql('''
    SELECT * 
    FROM parking_locations 
''')

# Featurize parking transactions

In order to build a regression model we need to featurize our dataset a bit. In this simple apporach our only variable is going to be the average occupancy at a parking spot at 1 hour prior to the transaction.

But first, let's remove outliers: transactions that reported higher occupancy than available parking spots.

In [ ]:
def calculate_time_deltas(df, delta, time_signature):
    return df['OccupancyDateTime'] + np.timedelta64(delta, time_signature)

def mark_outliers(df):
    return df['PaidOccupancy'] <= df['ParkingSpaceCount']

parking_transactions = bc.sql('''
    SELECT * 
    FROM parking_transactions 
    WHERE OccupancyDateTime >= DATE'2019-06-23'
''')
parking_transactions['time_prior_1h'] = calculate_time_deltas(parking_transactions, -1, 'h')
parking_transactions['not_outlier'] = mark_outliers(parking_transactions.merge(parking_locations[['SourceElementKey', 'ParkingSpaceCount']]))

parking_transactions = parking_transactions.query('not_outlier')
bc.create_table('parking_transactions', parking_transactions)

print(f'Number of transactions without outliers: {len(parking_transactions):,}')

Next, let's create a table with average spot occupancy at any given hour.

In [ ]:
parking_transactions_agg = bc.sql('''
    SELECT SourceElementKey
        , transaction_year
        , transaction_month
        , transaction_day
        , transaction_hour
        , AVG(CAST(PaidOccupancy AS FLOAT)) AS average_occupancy
    FROM (
        SELECT A.*
            , YEAR(OccupancyDateTime) AS transaction_year 
            , MONTH(OccupancyDateTime) AS transaction_month
            , DAYOFMONTH(OccupancyDateTime) AS transaction_day
            , HOUR(OccupancyDateTime) AS transaction_hour
        FROM parking_transactions AS A
    ) AS outer_query
    GROUP BY SourceElementKey
        , transaction_year
        , transaction_month
        , transaction_day
        , transaction_hour
    ''')

bc.create_table('parking_transactions_agg', parking_transactions_agg)

Finally, we can join the aggregates with the `parking_transactions` table.

In [ ]:
dataset_for_training = bc.sql('''
    WITH temp_query_prior_1h AS (
        SELECT A.SourceElementKey
            , A.PaidOccupancy AS Label
            , A.OccupancyDateTime
            , B.average_occupancy as AvgOccupancy_prior_1h
        FROM (
            SELECT SourceElementKey
                , PaidOccupancy
                , OccupancyDateTime
                , YEAR(time_prior_1h) AS transaction_year 
                , MONTH(time_prior_1h) AS transaction_month
                , DAYOFMONTH(time_prior_1h) AS transaction_day
                , HOUR(time_prior_1h) AS transaction_hour
            FROM parking_transactions) AS A
        LEFT OUTER JOIN parking_transactions_agg AS B
            ON A.SourceElementKey = B.SourceElementKey
                AND A.transaction_year = B.transaction_year
                AND A.transaction_month = B.transaction_month
                AND A.transaction_day = B.transaction_day
                AND A.transaction_hour = B.transaction_hour
    )
    SELECT hr_1.* 
    FROM temp_query_prior_1h AS hr_1
    
''')

In [ ]:
dataset_for_training.head()

We no longer will need the `SourceElementKey` nor the `OccupancyDateTime` columns so let's drop them. Also, there may be some `N\A`s so we can use `.dropna()` to remove them.

In [ ]:
dataset_for_training = (
    dataset_for_training[[
        'Label'
        , 'AvgOccupancy_prior_1h'
    ]]
    .dropna()
)
dataset_for_training.head()

In [ ]:
#### CLEAN UP TO FREE UP SOME MEMORY
bc.drop_table('parking_transactions')
del parking_transactions

## Build regression model

In this step we will build a Random Forest Regression model to predict the parking spot occupancy.

In [ ]:
from cuml.preprocessing import train_test_split

First, let's split the dataset into 4 subsets: training-features, training-label, testing-features, and testing-label.

In [ ]:
dataset_for_training['Label'] = dataset_for_training['Label'].astype('float32')

train_X, test_X, train_y, test_y = train_test_split(
    dataset_for_training[['AvgOccupancy_prior_1h']]
    , dataset_for_training['Label']
    , train_size=0.7
)

In [ ]:
print(f'Full size: {len(dataset_for_training):,}, size of training: {len(train_y):,}, size of testing: {len(test_y):,}')

In [ ]:
train_X.head()

Now it's time to build our model.

In [ ]:
from cuml import RandomForestRegressor
from cuml.metrics.regression import r2_score

In [ ]:
rfr = RandomForestRegressor(
    n_estimators=10
    , max_depth=10
    , min_rows_per_node=100
    , verbose=2
)

In [ ]:
rfr.fit(train_X, train_y)

In [ ]:
rfr.predict(test_X)

In [ ]:
print(f'R-squered score for our model: {r2_score(test_y, rfr.predict(test_X))}')

# Build a graph or roads in Seattle

In what follows we will build and traverse a graph of roads in Seattle.

But first we need to extract the `Lat` and `Lon` from the string column.

In [ ]:
def extractLon(location):
    lon = location.str.extract('([0-9\.\-]+) ([0-9\.]+)')[0]
    return lon

def extractLat(location):
    lon = location.str.extract('([0-9\.\-]+) ([0-9\.]+)')[1]
    return lon
    
parking_locations['longitude'] = extractLon(parking_locations['Location']).astype('float32')
parking_locations['latitude'] = extractLat(parking_locations['Location']).astype('float32')

parking_locations[['Location', 'longitude', 'latitude']].head()

## As crow flies vs as people walk

Previously, our approach assumed we could *fly*. More to it, we could fly through buildings! However, we know that's *normally* not true. So, in what follows we will use a graph of road intersections and road lengths kindly donated by John Murray from Fusion Data Science in an attempt to provide a more realistic way of walking.  

![fff](https://miro.medium.com/max/1400/1*zmMox5tjq93AiBZdPRG3gg.png)

#### Let's read the King County road data

In [ ]:
king_county_dir = 's3://bsql/data/seattle_parking/king_county_road_graph/'

road_graph_data = cudf.read_csv(
    f'{king_county_dir}king_county_road_graph_20190909.csv'
    , storage_options={'anon': True}
)
road_graph_data['node1'] = road_graph_data['node1'].astype('int32')
road_graph_data['node2'] = road_graph_data['node2'].astype('int32')
road_graph_data['LENGTH'] = road_graph_data['LENGTH'] * 3 # convert to feet as the LENGHT was given in yards

In [ ]:
road_nodes = cudf.read_csv(
    f'{king_county_dir}king_county_road_nodes_20190909.csv'
    , storage_options={'anon': True}
)
road_nodes['NodeID'] = road_nodes['NodeID'].astype('int32')

Store the maximum of the `NodeID` so we can later append the additional nodes that will be perpendicular to the actual parking locations. We also specify the offset - this will be used to append parking nodes.

In [ ]:
offset = 100000
nodeId = road_nodes['NodeID'].max()                       ## so we can number the parking nodes properly (since we'll be adding a perpendicular projections)
parking_nodes_idx = road_nodes['NodeID'].max() + offset   ## retain it so we can later filter the results to only parking locations
nodeId

Move all the parking locations to host (via `.to_pandas()` method) so we can loop through all the ~1500 parking locations. Here, we also create an empty DataFrame that will hold the parking location nodes.

In [ ]:
locations = parking_locations.to_pandas().to_dict('records')
parking_locations_nodes = cudf.DataFrame(columns=['NodeID', 'Lon', 'Lat', 'SourceElementKey'])
added_location_edges    = cudf.DataFrame(columns=['node1', 'node2', 'LENGTH'])

Let's process the parking data. The kernel below finds equations of two lines:

1. Line that goes through road intersections
2. Line that is perpendicular to (1) and goes through the parking location.

![lll](https://miro.medium.com/max/1296/1*4Sg3alMbrT3DndIzM9dS4Q.gif)

Ultimately, we are finind the intersection of these two lines -- we call it the `PROJ` point below.

In [ ]:
def kernel_find_projection(Lon_x, Lat_x, Lon_y, Lat_y, Lon_PROJ, Lat_PROJ, Lon_REF, Lat_REF):
    for i, (lon_x, lat_x, lon_y, lat_y) in enumerate(zip(Lon_x, Lat_x, Lon_y, Lat_y)):
        # special case where A and B have the same LON i.e. vertical line
        if lon_x == lon_y:
            Lon_PROJ[i] = lon_x
            Lat_PROJ[i] = Lat_REF    
        else:
            # find slope
            a_xy = (lat_x - lat_y) / float(lon_x - lon_y)

            # special case where A and B have the same LAT i.e. horizontal line
            if a_xy == 0:
                Lon_PROJ[i] = Lon_REF
                Lat_PROJ[i] = lat_x
            else: 
                # if neither of the above special cases apply
                # find the equation of the perpendicular line
                a_R  = -1 / a_xy                    ### SLOPE

                # find intersections
                b_xy = lat_x - a_xy * lon_x
                b_R  = Lat_REF - a_R  * Lon_REF

                # find the coordinates
                Lon_PROJ[i] = (b_R - b_xy) / (a_xy - a_R)
                Lat_PROJ[i] = a_R * Lon_PROJ[i] + b_R

In [ ]:
%%time
parking_locations_cnt = len(locations)
print('Number of parking locations: {0:,}'.format(parking_locations_cnt))

for i, loc in enumerate(locations):
    if i % 100 == 0:
        print('Processed: {0:,} ({1:.2%}) nodes'.format(i, i/float(parking_locations_cnt)))
        
    #### INCREASE THE COUNTER AND GET THE REFERENCE POINT
    nodeId = nodeId + 1
    lat_r = loc['latitude']
    lon_r = loc['longitude']

    #### APPEND GEO COORDINATES TO INTERSECTION AND SUBSET DOWN THE DATASET
    #### TO POINTS WITHIN ~2000ft FROM PARKING SPOT
    paths = (
        road_graph_data
        .rename(columns={'node1': 'NodeID'})
        .merge(road_nodes[['NodeID', 'Lat', 'Lon']], on='NodeID', how='left')
        .rename(columns={'NodeID': 'node1', 'node2': 'NodeID'})
        .merge(road_nodes[['NodeID', 'Lat', 'Lon']], on='NodeID', how='left')
        .rename(columns={'NodeID': 'node2'})
        .query('Lat_x >= (@lat_r - 0.005) and Lat_x <= (@lat_r + 0.005)')
        .query('Lon_x >= (@lon_r - 0.005) and Lon_x <= (@lon_r + 0.005)')
        .query('Lat_y >= (@lat_r - 0.005) and Lat_y <= (@lat_r + 0.005)')
        .query('Lon_y >= (@lon_r - 0.005) and Lon_y <= (@lon_r + 0.005)')
    )

    #### APPEND THE PARKING LOCATION SO WE CAN CALCULATE DISTANCES
    paths['Lon_REF'] = loc['longitude']
    paths['Lat_REF'] = loc['latitude']

    paths = paths.apply_rows(
        kernel_find_projection
        , incols  = ['Lon_x', 'Lat_x', 'Lon_y', 'Lat_y', 'Lon_REF', 'Lat_REF']
        , outcols = {'Lon_PROJ': np.float64, 'Lat_PROJ': np.float64}
        , kwargs  = {'Lon_REF': loc['longitude'], 'Lat_REF': loc['latitude']}
    )

    #### CALCULATE THE DISTANCES SO WE CAN CHECK IF THE PROJ POINT IS BETWEEN ROAD NODES
    paths['Length_x_PROJ'] = cuspatial.haversine_distance(
              paths['Lon_x']
            , paths['Lat_x']
            , paths['Lon_PROJ']
            , paths['Lat_PROJ'])# * 0.621371 * 5280
    paths['Length_x_PROJ'] = paths['Length_x_PROJ'] * 0.621371 * 5280

    paths['Length_y_PROJ'] = cuspatial.haversine_distance(
              paths['Lon_y']
            , paths['Lat_y']
            , paths['Lon_PROJ']
            , paths['Lat_PROJ']) 
    paths['Length_y_PROJ'] = paths['Length_y_PROJ'] * 0.621371 * 5280

    paths['Length_REF_PROJ'] = cuspatial.haversine_distance(
              paths['Lon_REF']
            , paths['Lat_REF']
            , paths['Lon_PROJ']
            , paths['Lat_PROJ']) 
    paths['Length_REF_PROJ'] = paths['Length_REF_PROJ'] * 0.621371 * 5280

    #### SELECT THE POINTS THAT A LESS THAN OR EQAL TO TOTAL LENGTH OF THE EDGE (WITHIN 1 ft)
    paths['PROJ_between'] = (paths['Length_x_PROJ'] + paths['Length_y_PROJ']) <= (paths['LENGTH'] + 1)
    
    #### SELECT THE CLOSEST
    closest = (
        paths
        .query('PROJ_between')
        .nsmallest(1, 'Length_REF_PROJ')
        .to_pandas()
        .to_dict('records')[0]
    )

    # add nodes
    nodes =    cudf.DataFrame({
          'NodeID': [nodeId + offset, nodeId]
        , 'Lon':    [closest['Lon_REF'], closest['Lon_PROJ']]
        , 'Lat':    [closest['Lat_REF'], closest['Lat_PROJ']]
        , 'SourceElementKey': [loc['SourceElementKey'], None]
    })

    parking_locations_nodes = cudf.concat([parking_locations_nodes, nodes])

    # add edges (bi-directional)
    edges = cudf.DataFrame({
          'node1':  [nodeId, nodeId, nodeId, closest['node1'], closest['node2'], nodeId + offset]
        , 'node2':  [closest['node1'], closest['node2'], nodeId + offset, nodeId, nodeId, nodeId]
        , 'LENGTH': [
              closest['Length_x_PROJ'], closest['Length_y_PROJ'], closest['Length_REF_PROJ']
            , closest['Length_x_PROJ'], closest['Length_y_PROJ'], closest['Length_REF_PROJ']
        ]
    })

    added_location_edges = cudf.concat([added_location_edges, edges]) ## append to the temp DataFrame

print('Finished processing...')

In [ ]:
parking_locations_nodes.head()

In [ ]:
road_graph_data.head()

In [ ]:
road_nodes = (
    cudf
    .concat([road_nodes[['NodeID', 'Lon', 'Lat']], parking_locations_nodes])
    .reset_index(drop=True)
)

Now we can find the nearest intersections from the Space Needle!

In [ ]:
location = {'latitude': 47.620422, 'longitude': -122.349358}

In [ ]:
road_nodes['Lon_REF'] = location['longitude']
road_nodes['Lat_REF'] = location['latitude']

road_nodes['Distance'] = cuspatial.haversine_distance(
          road_nodes['Lon']
        , road_nodes['Lat']
        , road_nodes['Lon_REF']
        , road_nodes['Lat_REF']) 
road_nodes['Distance'] = road_nodes['Distance'] * 0.621371 * 5280

space_needle_to_nearest_intersection = road_nodes.nsmallest(5, 'Distance') ### Space Needle is surrounded by around 5 road intersections hence we add 5
space_needle_to_nearest_intersection_dist = space_needle_to_nearest_intersection['Distance'].to_array()[0]

space_needle_to_nearest_intersection['node1'] = nodeId + 2
space_needle_to_nearest_intersection = (
    space_needle_to_nearest_intersection
    .rename(columns={'NodeID': 'node2', 'Distance': 'LENGTH'})
    [['node1', 'node2', 'LENGTH']]
)

road_graph_data = cudf.concat([space_needle_to_nearest_intersection, added_location_edges, road_graph_data])
space_needle_to_nearest_intersection ### SHOW THE EDGES

### The road graph

In [ ]:
road_graph_data = road_graph_data.reset_index(drop=True)
road_graph_data['node1'] = road_graph_data['node1'].astype('int32')
road_graph_data['node2'] = road_graph_data['node2'].astype('int32')

In [ ]:
g = cugraph.Graph()
g.from_cudf_edgelist(
    road_graph_data
    , source='node1'
    , destination='node2'
    , edge_attr='LENGTH'
    , renumber=False
)

Now we can use the `.sssp(...)` method from `cugraph` to find the shortest distances to parking spots from the Space Needle!

In [ ]:
all_distances = cugraph.sssp(g, nodeId + 2)
distances = all_distances.query('vertex > @parking_nodes_idx and distance < 1000')
distances

### Use the regression model

To use our regression model, let's retrieve the `1hr` feature for all the parking spots from the previous step.

In [ ]:
date = dttm.datetime.strptime('2019-06-24 13:21:00', '%Y-%m-%d %H:%M:%S')

inference = distances.merge(parking_locations_nodes, left_on='vertex', right_on='NodeID')
inference['OccupancyDateTime'] = date
inference['time_prior_1h'] = date + dttm.timedelta(hours=-1)
bc.create_table('inference', inference)
inference

In [ ]:
inference = bc.sql(f'''
    WITH temp_query_prior_1h AS (
        SELECT A.SourceElementKey
            , A.OccupancyDateTime
            , B.average_occupancy as AvgOccupancy_prior_1h
        FROM (
            SELECT SourceElementKey
                , OccupancyDateTime
                , YEAR(time_prior_1h) AS transaction_year 
                , MONTH(time_prior_1h) AS transaction_month
                , DAYOFMONTH(time_prior_1h) AS transaction_day
                , HOUR(time_prior_1h) AS transaction_hour
            FROM inference) AS A
        LEFT OUTER JOIN parking_transactions_agg AS B
            ON A.SourceElementKey = B.SourceElementKey
                AND A.transaction_year = B.transaction_year
                AND A.transaction_month = B.transaction_month
                AND A.transaction_day = B.transaction_day
                AND A.transaction_hour = B.transaction_hour
    )
    SELECT hr_1.* 
    FROM temp_query_prior_1h AS hr_1
    ORDER BY SourceElementKey
    
''')
inference.head()

In [ ]:
inference_X = (
    inference[[
        'SourceElementKey'
        , 'AvgOccupancy_prior_1h'
    ]]
    .merge(parking_locations[['SourceElementKey']], on=['SourceElementKey'])
    .sort_values(by='SourceElementKey')
    .dropna()
    .drop(columns=['SourceElementKey'])
)
inference_X

Now we can run the prediction.

In [ ]:
inference.join(rfr.predict(inference_X).to_frame('prediction'))

### Stepping through the graph

`cugraph` returns a DataFrame with vertex, distance to that vertex, and the total distance traveled to that vertex from the `nodeId + 1` node -- the Space Needle. Here, we unfold the full path.

In [ ]:
# unfold -- create the whole path
closest_node = nodeId + 2
parking_cnt = distances['vertex'].count()

for i in range(parking_cnt):
    print('Processing record: {0}'.format(i))
    parking_node = distances.iloc[i]

    vertex = int(parking_node['vertex'])
    predecessor = int(parking_node['predecessor'])
    
    if i == 0:
        paths = all_distances.query('vertex == @vertex')
    else:
        paths = cudf.concat([all_distances.query('vertex == @vertex'), paths])

    while vertex != closest_node:
        temp = all_distances.query('vertex == @predecessor')
        paths = cudf.concat([temp, paths])
        predecessor = temp['predecessor'].to_array()[0]
        vertex = temp['vertex'].to_array()[0]

In [ ]:
paths

If you were to plot these, this is roughtly what you would get
![map](https://miro.medium.com/max/1400/1*UeOfPlEjzyRt-gxUHauUIw.png)